In [1]:
from __future__ import (division, unicode_literals, absolute_import,
                        print_function)

from ema_workbench import (Model, RealParameter, Constant, CategoricalParameter, 
                           TimeSeriesOutcome, ScalarOutcome, ema_logging, perform_experiments, 
                           MultiprocessingEvaluator, Policy)
from ema_workbench.connectors.vensim import (VensimModel , VensimModelStructureInterface, set_value)
from ema_workbench.em_framework.samplers import sample_levers
from ema_workbench.util.utilities import save_results
from ema_workbench import load_results
from ema_workbench.analysis import pairs_plotting
from ema_workbench.analysis.pairs_plotting import (pairs_lines, pairs_scatter)
from ema_workbench.analysis import clusterer, plotting, Density, prim
from ema_workbench.analysis.plotting import lines, envelopes, kde_over_time, multiple_densities



import timeit
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

from ema_workbench.analysis.plotting import lines

C:\Programs\Anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:27: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)
C:\Programs\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Programs\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Programs\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Programs\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath

# Select vensim file

In [2]:
wd = r'./VENSIM' # Directory of the vensim model
model_file=r'Maasmodel_v075.vpmx' # Name of the vensim model

# General manual input

In [3]:
#FILL IN THIS CELL
#WEIR
Crest_level_weir1 = 8.35
Crest_level_weir2 = 7.15
Crest_level_weir3 = 6.9
Crest_level_weir4 = 7.2
Crest_level_weir5 = 8.4
Crest_level_weir6 = 10.9
Location_weir1    = 57500
Location_weir2    = 69500
Location_weir3    = 87500
Location_weir4    = 135500
Location_weir5    = 163500
Location_weir6    = 189500
Leakage_low_weir1    = 1.2e6
Leakage_low_weir2    = 1e6
Leakage_low_weir3    = 1e6
Leakage_low_weir4    = 3e6
Leakage_low_weir5    = 3e6
Leakage_low_weir6    = 2e6
Leakage_up_weir1     = 6e6
Leakage_up_weir2     = 5e6
Leakage_up_weir3     = 5e6
Leakage_up_weir4     = 9e6
Leakage_up_weir5     = 9e6
Leakage_up_weir6     = 8e6

#LOCK
Intensity_commercial_lock1 = 44
Intensity_commercial_lock2 = 62
Intensity_commercial_lock3 = 439
Intensity_commercial_lock4 = 547
Intensity_commercial_lock5 = 289
Intensity_commercial_lock6 = 334
Intensity_leisure_lock1 = 643
Intensity_leisure_lock2 = 518
Intensity_leisure_lock3 = 458
Intensity_leisure_lock4 = 498
Intensity_leisure_lock5 = 511
Intensity_leisure_lock6 = 807

In [4]:
#NAME CLIMATE SCENARIOS
CS1 = 'Climate scenario GL'
CS2 = 'Climate scenario WH'

#SAVE FIGURE
save = False
fig_size = (10,7)
dpi = 300
%matplotlib inline

#Starting year
start_year = 2020
end_year   = 2100


#100 years
years = np.linspace(start_year,end_year,81)

#SECONDS TO Week
SEC_WEEK = 60*60*24*7 

#Different draught durations in weeks
Draught_durations = [4,6,8,10,12,14,16] #Duur van de droogte periode variërend van 4 tot 16 weken

#Different draught intensities in m3/s
Draught_intensities = [20,25,30,35,40,45,1000] #Maximale afvoer in periodes van droogte, 1000 staat voor geen droogte periode 

#Distance from inflow
distance_inflow = np.linspace(500, 189500, 190)

#100 year discharge series
year_100_dischargeseries = list(range(1911,2015))

# Model parameters

In [5]:
# turn on logging
ema_logging.log_to_stderr(ema_logging.INFO)

# instantiate a model
vensimModel = VensimModel('MeuseModel', wd=wd, model_file=model_file)
    
# (X) External factors and States of the World
vensimModel.uncertainties = [CategoricalParameter('Climate scenario', [1, 2]),
                             CategoricalParameter('Socio Economic scenario', [1, 2]),
                           
#Uncertain model variables  
                             CategoricalParameter('Draught period intensity', Draught_intensities ),
                             CategoricalParameter('Draught period length', Draught_durations),
                             CategoricalParameter('Simulated year', years),
                             RealParameter('Water loss one levelling cycle', 6000, 12000),
                             RealParameter('Extra shipping intensity Diked Meuse', 0.2, 0.5),
                             RealParameter('Percentage of ships that are class IV', 0.2, 0.3),
                             RealParameter('Percentage of ships that are class V', 0.4, 0.5),
                             RealParameter('Percentage transport via water', 0.1, 0.3),
                             RealParameter('Leakage weir and lock[Weir1]', Leakage_low_weir1, Leakage_up_weir1),
                             RealParameter('Leakage weir and lock[Weir2]', Leakage_low_weir2, Leakage_up_weir2),
                             RealParameter('Leakage weir and lock[Weir3]', Leakage_low_weir3, Leakage_up_weir3),
                             RealParameter('Leakage weir and lock[Weir4]', Leakage_low_weir4, Leakage_up_weir4),
                             RealParameter('Leakage weir and lock[Weir5]', Leakage_low_weir5, Leakage_up_weir5),
                             RealParameter('Leakage weir and lock[Weir6]', Leakage_low_weir6, Leakage_up_weir6),
                             CategoricalParameter('Year from discharge series', year_100_dischargeseries)
                                                          ]
                                                  
                                           


# (R) Other system parameters
vensimModel.constants = [#Water flow
                         Constant('Closing retaining lock Limmel',  1200  ),
                         Constant('Discharge to turn on pumps Maasbracht',  50  ),
                         Constant('Dryest time of the year', 26),
                         Constant('Grid cell length', 1000),
                         Constant('Initial equilibrium depth', 3.5),
                         Constant('Low discharge', 50*SEC_WEEK),
                         Constant('Maximum critical depth', 1.5),
                         Constant('Maximum water depth', 0.2),
                         Constant('Minimum width', 1),
                         Constant('Reference year', 2014),
                         Constant('Water loss due to levelling at Maasbracht', 15*SEC_WEEK),
                         Constant('Water loss Weurt', 0.8*SEC_WEEK),
    
                        

                         #Shipping flow
                         Constant('Commercial shipping growth WLOL', -0.00041675 ),
                         Constant('Commercial shipping growth WLOH', 0.000833  ),
                         Constant('Draught ship of CEMT class III', 2.7),
                         Constant('Draught ship of CEMT class IV', 3),
                         Constant('Draught ship of CEMT class V', 3.5),
                         Constant('Intensity commercial shipping reference year[Weir1]', Intensity_commercial_lock1 ),
                         Constant('Intensity commercial shipping reference year[Weir2]', Intensity_commercial_lock2 ),
                         Constant('Intensity commercial shipping reference year[Weir3]', Intensity_commercial_lock3 ),
                         Constant('Intensity commercial shipping reference year[Weir4]', Intensity_commercial_lock4 ),
                         Constant('Intensity commercial shipping reference year[Weir5]', Intensity_commercial_lock5 ),
                         Constant('Intensity commercial shipping reference year[Weir6]', Intensity_commercial_lock6 ),
                         Constant('Intensity leisure shipping reference year[Weir1]', Intensity_leisure_lock1),
                         Constant('Intensity leisure shipping reference year[Weir2]', Intensity_leisure_lock2),
                         Constant('Intensity leisure shipping reference year[Weir3]', Intensity_leisure_lock3),
                         Constant('Intensity leisure shipping reference year[Weir4]', Intensity_leisure_lock4),
                         Constant('Intensity leisure shipping reference year[Weir5]', Intensity_leisure_lock5),
                         Constant('Intensity leisure shipping reference year[Weir6]', Intensity_leisure_lock6),
                         Constant('Leisure shipping growth WLOL', -0.000275 ),
                         Constant('Leisure shipping growth WLOH', -0.0015 ),
                         Constant('Large lock factor', 1.75),
                         Constant('Middle sized lock factor', 1.4),
                         Constant('Small lock factor', 1),
                         Constant('Max number of ships in small chamber', 4),
                         Constant('Minimum percentage of ships sailing over the river Meuse', 0.1),
                         Constant('Percentage transport via water reference year', 0.2),
                         Constant('Sill height in front of lock[Weir1]', 4.1),
                         Constant('Sill height in front of lock[Weir2]', 2.2),
                         Constant('Sill height in front of lock[Weir3]', 1.25),
                         Constant('Sill height in front of lock[Weir4]', 0.1),
                         Constant('Sill height in front of lock[Weir5]', 1.5),
                         Constant('Sill height in front of lock[Weir6]', 3),
                         Constant('Time sailing in to lock', 10),
                         Constant('Time sailing out of lock', 10),
                         Constant('Time securing the ship', 5),
    
    
                         #Fresh water buffer
                         Constant('Original crest height[Weir1]', 8.35),
                         Constant('Original crest height[Weir1]', 7.15),                         
                         Constant('Original crest height[Weir1]', 6.9),
                         Constant('Original crest height[Weir1]', 7.2),
                         Constant('Original crest height[Weir1]', 8.4),
                         Constant('Original crest height[Weir1]', 8.5),
    
    
    

                         #BASE ENSEMBLE
                         Constant('Crest level weir[Weir1]', 8.35),
                         Constant('Crest level weir[Weir2]', 7.15),
                         Constant('Crest level weir[Weir3]', 6.9),
                         Constant('Crest level weir[Weir4]', 7.2),
                         Constant('Crest level weir[Weir5]', 8.4),
                         Constant('Crest level weir[Weir6]', 8.5),
                         Constant('Weir location[Weir1]', 57500),
                         Constant('Weir location[Weir2]', 69500),
                         Constant('Weir location[Weir3]', 87500),
                         Constant('Weir location[Weir4]', 136500),
                         Constant('Weir location[Weir5]', 163500),
                         Constant('Weir location[Weir6]', 189500),
                         Constant('Pump switch', 0),
                         Constant('Levelling restrictions switch', 0),
                         Constant('Discharge to open weir[Weir1]', 1350),
                         Constant('Discharge to open weir[Weir2]', 1100),
                         Constant('Discharge to open weir[Weir3]', 1000),
                         Constant('Discharge to open weir[Weir4]', 1300),
                         Constant('Discharge to open weir[Weir5]', 1200),
                         Constant('Discharge to open weir[Weir6]', 1200),
                         Constant('Number of small chambers[Weir1]', 0),
                         Constant('Number of small chambers[Weir2]', 0),
                         Constant('Number of small chambers[Weir3]', 2),
                         Constant('Number of small chambers[Weir4]', 2),
                         Constant('Number of small chambers[Weir5]', 1),
                         Constant('Number of small chambers[Weir6]', 1),
                         Constant('Number of middle sized chambers[Weir1]', 0),
                         Constant('Number of middle sized chambers[Weir2]', 0),  
                         Constant('Number of middle sized chambers[Weir3]', 0),  
                         Constant('Number of middle sized chambers[Weir4]', 0),  
                         Constant('Number of middle sized chambers[Weir5]', 0),  
                         Constant('Number of middle sized chambers[Weir6]', 1),                       
                         Constant('Number of large chambers[Weir1]', 1),                      
                         Constant('Number of large chambers[Weir2]', 1),                      
                         Constant('Number of large chambers[Weir3]', 1),                      
                         Constant('Number of large chambers[Weir4]', 1),                      
                         Constant('Number of large chambers[Weir5]', 0),                      
                         Constant('Number of large chambers[Weir6]', 0)
                         ]

                       

                        
    
    
    
# (M) Performance Metrics and Outcomes of Interest
vensimModel.outcomes = [TimeSeriesOutcome('Percentage of full fresh water buffer[Weir1]'),
                        TimeSeriesOutcome('Percentage of full fresh water buffer[Weir2]'),
                        TimeSeriesOutcome('Percentage of full fresh water buffer[Weir3]'),
                        TimeSeriesOutcome('Percentage of full fresh water buffer[Weir4]'),
                        TimeSeriesOutcome('Percentage of full fresh water buffer[Weir5]'),
                        TimeSeriesOutcome('Percentage of full fresh water buffer[Weir6]'),
                        TimeSeriesOutcome('Average velocity through weir section[Weir1]'),
                        TimeSeriesOutcome('Average velocity through weir section[Weir2]'),
                        TimeSeriesOutcome('Average velocity through weir section[Weir3]'),
                        TimeSeriesOutcome('Average velocity through weir section[Weir4]'),
                        TimeSeriesOutcome('Average velocity through weir section[Weir5]'),
                        TimeSeriesOutcome('Average velocity through weir section[Weir6]'),
                        TimeSeriesOutcome('"I/C value at lock"[Weir1]'),
                        TimeSeriesOutcome('"I/C value at lock"[Weir2]'),
                        TimeSeriesOutcome('"I/C value at lock"[Weir3]'),
                        TimeSeriesOutcome('"I/C value at lock"[Weir4]'),
                        TimeSeriesOutcome('"I/C value at lock"[Weir5]'),
                        TimeSeriesOutcome('"I/C value at lock"[Weir6]'),
                        TimeSeriesOutcome('Days of free flow'),
                        TimeSeriesOutcome('Water depth[S1]'),
                        TimeSeriesOutcome('Water depth[S2]'),
                        TimeSeriesOutcome('Water depth[S3]'),
                        TimeSeriesOutcome('Water depth[S4]'),
                        TimeSeriesOutcome('Water depth[S5]'),
                        TimeSeriesOutcome('Water depth[S6]'),
                        TimeSeriesOutcome('Water depth[S7]'),
                        TimeSeriesOutcome('Water depth[S8]'),
                        TimeSeriesOutcome('Water depth[S9]'),
                        TimeSeriesOutcome('Water depth[S10]'),
                        TimeSeriesOutcome('Water depth[S11]'),
                        TimeSeriesOutcome('Water depth[S12]'),
                        TimeSeriesOutcome('Water depth[S13]'),
                        TimeSeriesOutcome('Water depth[S14]'),
                        TimeSeriesOutcome('Water depth[S15]'),
                        TimeSeriesOutcome('Water depth[S16]'),
                        TimeSeriesOutcome('Water depth[S17]'),
                        TimeSeriesOutcome('Water depth[S18]'),
                        TimeSeriesOutcome('Water depth[S19]'),
                        TimeSeriesOutcome('Water depth[S20]'),
                        TimeSeriesOutcome('Water depth[S21]'),
                        TimeSeriesOutcome('Water depth[S22]'),
                        TimeSeriesOutcome('Water depth[S23]'),
                        TimeSeriesOutcome('Water depth[S24]'),
                        TimeSeriesOutcome('Water depth[S25]'),
                        TimeSeriesOutcome('Water depth[S26]'),
                        TimeSeriesOutcome('Water depth[S27]'),
                        TimeSeriesOutcome('Water depth[S28]'),
                        TimeSeriesOutcome('Water depth[S29]'),
                        TimeSeriesOutcome('Water depth[S30]'),
                        TimeSeriesOutcome('Water depth[S31]'),
                        TimeSeriesOutcome('Water depth[S32]'),
                        TimeSeriesOutcome('Water depth[S33]'),
                        TimeSeriesOutcome('Water depth[S34]'),
                        TimeSeriesOutcome('Water depth[S35]'),
                        TimeSeriesOutcome('Water depth[S36]'),
                        TimeSeriesOutcome('Water depth[S37]'),
                        TimeSeriesOutcome('Water depth[S38]'),
                        TimeSeriesOutcome('Water depth[S39]'),
                        TimeSeriesOutcome('Water depth[S40]'),
                        TimeSeriesOutcome('Water depth[S41]'),
                        TimeSeriesOutcome('Water depth[S42]'),
                        TimeSeriesOutcome('Water depth[S43]'),
                        TimeSeriesOutcome('Water depth[S44]'),
                        TimeSeriesOutcome('Water depth[S45]'),
                        TimeSeriesOutcome('Water depth[S46]'),
                        TimeSeriesOutcome('Water depth[S47]'),
                        TimeSeriesOutcome('Water depth[S48]'),
                        TimeSeriesOutcome('Water depth[S49]'),
                        TimeSeriesOutcome('Water depth[S50]'),
                        TimeSeriesOutcome('Water depth[S51]'),
                        TimeSeriesOutcome('Water depth[S52]'),
                        TimeSeriesOutcome('Water depth[S53]'),
                        TimeSeriesOutcome('Water depth[S54]'),
                        TimeSeriesOutcome('Water depth[S55]'),
                        TimeSeriesOutcome('Water depth[S56]'),
                        TimeSeriesOutcome('Water depth[S57]'),
                        TimeSeriesOutcome('Water depth[S58]'),
                        TimeSeriesOutcome('Water depth[S59]'),
                        TimeSeriesOutcome('Water depth[S60]'),
                        TimeSeriesOutcome('Water depth[S61]'),
                        TimeSeriesOutcome('Water depth[S62]'),
                        TimeSeriesOutcome('Water depth[S63]'),
                        TimeSeriesOutcome('Water depth[S64]'),
                        TimeSeriesOutcome('Water depth[S65]'),
                        TimeSeriesOutcome('Water depth[S66]'),
                        TimeSeriesOutcome('Water depth[S67]'),
                        TimeSeriesOutcome('Water depth[S68]'),
                        TimeSeriesOutcome('Water depth[S69]'),
                        TimeSeriesOutcome('Water depth[S70]'),
                        TimeSeriesOutcome('Water depth[S71]'),
                        TimeSeriesOutcome('Water depth[S72]'),
                        TimeSeriesOutcome('Water depth[S73]'),
                        TimeSeriesOutcome('Water depth[S74]'),
                        TimeSeriesOutcome('Water depth[S75]'),
                        TimeSeriesOutcome('Water depth[S76]'),
                        TimeSeriesOutcome('Water depth[S77]'),
                        TimeSeriesOutcome('Water depth[S78]'),
                        TimeSeriesOutcome('Water depth[S79]'),
                        TimeSeriesOutcome('Water depth[S80]'),
                        TimeSeriesOutcome('Water depth[S81]'),
                        TimeSeriesOutcome('Water depth[S82]'),
                        TimeSeriesOutcome('Water depth[S83]'),
                        TimeSeriesOutcome('Water depth[S84]'),
                        TimeSeriesOutcome('Water depth[S85]'),
                        TimeSeriesOutcome('Water depth[S86]'),
                        TimeSeriesOutcome('Water depth[S87]'),
                        TimeSeriesOutcome('Water depth[S88]'),
                        TimeSeriesOutcome('Water depth[S89]'),
                        TimeSeriesOutcome('Water depth[S90]'),
                        TimeSeriesOutcome('Water depth[S91]'),
                        TimeSeriesOutcome('Water depth[S92]'),
                        TimeSeriesOutcome('Water depth[S93]'),
                        TimeSeriesOutcome('Water depth[S94]'),
                        TimeSeriesOutcome('Water depth[S95]'),
                        TimeSeriesOutcome('Water depth[S96]'),
                        TimeSeriesOutcome('Water depth[S97]'),
                        TimeSeriesOutcome('Water depth[S98]'),
                        TimeSeriesOutcome('Water depth[S99]'),
                        TimeSeriesOutcome('Water depth[S100]'),
                        TimeSeriesOutcome('Water depth[S101]'),
                        TimeSeriesOutcome('Water depth[S102]'),                        
                        TimeSeriesOutcome('Water depth[S103]'),
                        TimeSeriesOutcome('Water depth[S104]'),                        
                        TimeSeriesOutcome('Water depth[S105]'),
                        TimeSeriesOutcome('Water depth[S106]'),
                        TimeSeriesOutcome('Water depth[S107]'),
                        TimeSeriesOutcome('Water depth[S108]'),                        
                        TimeSeriesOutcome('Water depth[S109]'),
                        TimeSeriesOutcome('Water depth[S110]'),
                        TimeSeriesOutcome('Water depth[S111]'),
                        TimeSeriesOutcome('Water depth[S112]'),                        
                        TimeSeriesOutcome('Water depth[S113]'),
                        TimeSeriesOutcome('Water depth[S114]'),                        
                        TimeSeriesOutcome('Water depth[S115]'),
                        TimeSeriesOutcome('Water depth[S116]'),
                        TimeSeriesOutcome('Water depth[S117]'),
                        TimeSeriesOutcome('Water depth[S118]'),                        
                        TimeSeriesOutcome('Water depth[S119]'),
                        TimeSeriesOutcome('Water depth[S120]'),
                        TimeSeriesOutcome('Water depth[S121]'),
                        TimeSeriesOutcome('Water depth[S122]'),                        
                        TimeSeriesOutcome('Water depth[S123]'),
                        TimeSeriesOutcome('Water depth[S124]'),                        
                        TimeSeriesOutcome('Water depth[S125]'),
                        TimeSeriesOutcome('Water depth[S126]'),
                        TimeSeriesOutcome('Water depth[S127]'),
                        TimeSeriesOutcome('Water depth[S128]'),                        
                        TimeSeriesOutcome('Water depth[S129]'),
                        TimeSeriesOutcome('Water depth[S130]'),
                        TimeSeriesOutcome('Water depth[S131]'),
                        TimeSeriesOutcome('Water depth[S132]'),                        
                        TimeSeriesOutcome('Water depth[S133]'),
                        TimeSeriesOutcome('Water depth[S134]'),                        
                        TimeSeriesOutcome('Water depth[S135]'),
                        TimeSeriesOutcome('Water depth[S136]'),
                        TimeSeriesOutcome('Water depth[S137]'),
                        TimeSeriesOutcome('Water depth[S138]'),                        
                        TimeSeriesOutcome('Water depth[S139]'),
                        TimeSeriesOutcome('Water depth[S140]'),
                        TimeSeriesOutcome('Water depth[S141]'),
                        TimeSeriesOutcome('Water depth[S142]'),                        
                        TimeSeriesOutcome('Water depth[S143]'),
                        TimeSeriesOutcome('Water depth[S144]'),                        
                        TimeSeriesOutcome('Water depth[S145]'),
                        TimeSeriesOutcome('Water depth[S146]'),
                        TimeSeriesOutcome('Water depth[S147]'),
                        TimeSeriesOutcome('Water depth[S148]'),                        
                        TimeSeriesOutcome('Water depth[S149]'),
                        TimeSeriesOutcome('Water depth[S150]'),
                        TimeSeriesOutcome('Water depth[S151]'),
                        TimeSeriesOutcome('Water depth[S152]'),                        
                        TimeSeriesOutcome('Water depth[S153]'),
                        TimeSeriesOutcome('Water depth[S154]'),                        
                        TimeSeriesOutcome('Water depth[S155]'),
                        TimeSeriesOutcome('Water depth[S156]'),
                        TimeSeriesOutcome('Water depth[S157]'),
                        TimeSeriesOutcome('Water depth[S158]'),                        
                        TimeSeriesOutcome('Water depth[S159]'),
                        TimeSeriesOutcome('Water depth[S160]'),
                        TimeSeriesOutcome('Water depth[S161]'),
                        TimeSeriesOutcome('Water depth[S162]'),                        
                        TimeSeriesOutcome('Water depth[S163]'),
                        TimeSeriesOutcome('Water depth[S164]'),                        
                        TimeSeriesOutcome('Water depth[S165]'),
                        TimeSeriesOutcome('Water depth[S166]'),
                        TimeSeriesOutcome('Water depth[S167]'),
                        TimeSeriesOutcome('Water depth[S168]'),                        
                        TimeSeriesOutcome('Water depth[S169]'),
                        TimeSeriesOutcome('Water depth[S170]'),
                        TimeSeriesOutcome('Water depth[S171]'),
                        TimeSeriesOutcome('Water depth[S172]'),                        
                        TimeSeriesOutcome('Water depth[S173]'),
                        TimeSeriesOutcome('Water depth[S174]'),                        
                        TimeSeriesOutcome('Water depth[S175]'),
                        TimeSeriesOutcome('Water depth[S176]'),
                        TimeSeriesOutcome('Water depth[S177]'),
                        TimeSeriesOutcome('Water depth[S178]'),                        
                        TimeSeriesOutcome('Water depth[S179]'),
                        TimeSeriesOutcome('Water depth[S180]'),
                        TimeSeriesOutcome('Water depth[S181]'),
                        TimeSeriesOutcome('Water depth[S182]'),                        
                        TimeSeriesOutcome('Water depth[S183]'),
                        TimeSeriesOutcome('Water depth[S184]'),                        
                        TimeSeriesOutcome('Water depth[S185]'),
                        TimeSeriesOutcome('Water depth[S186]'),
                        TimeSeriesOutcome('Water depth[S187]'),
                        TimeSeriesOutcome('Water depth[S188]'),                        
                        TimeSeriesOutcome('Water depth[S189]'),                        
                        TimeSeriesOutcome('Water depth[S190]')
                       ]

# Perform the experiments and save the results

In [6]:
nr_scenarios = 8
nr_policies = 0    

policies = sample_levers([vensimModel], nr_policies)
policies = [policy for policy in policies]

print('Simulating the model with', nr_scenarios, 'scenarios and', nr_policies, 'design decisions')


Simulating the model with 8 scenarios and 0 design decisions


In [7]:

start_time = timeit.default_timer()

results = perform_experiments(vensimModel, nr_scenarios, nr_policies)
#with MultiprocessingEvaluator(vensimModel) as evaluator:
#    results = evaluator.perform_experiments(scenarios=nr_scenarios, policies=nr_policies)


elapsed = timeit.default_timer() - start_time
print("Total time in minutes:", elapsed/60, "-- Time per run in seconds:", 
      elapsed/(nr_scenarios * max(len(policies),1)) ) 


[MainProcess/INFO] performing 8 scenarios * 1 policies * 1 model(s) = 8 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 1 cases completed
[MainProcess/INFO] 2 cases completed
[MainProcess/INFO] 3 cases completed
[MainProcess/INFO] 4 cases completed
[MainProcess/INFO] 5 cases completed
[MainProcess/INFO] 6 cases completed
[MainProcess/INFO] 7 cases completed
[MainProcess/INFO] 8 cases completed
[MainProcess/INFO] experiments finished


Total time in minutes: 0.63025427 -- Time per run in seconds: 4.726907025


Save results

In [8]:
experiments, outcomes = results

#save_results(results, r'C:\Users\douwe\OneDrive\Documenten\1.0 School\8.0 Thesis\5.0 Execution\Maasmodel\Results\BaseEnsemble_8000.tar')